In [ ]:
import requests
from sketch.examples.prompt_machine import *
import json
import os
import sqlite3
import random

PM_SETTINGS["VERBOSE"] = False
SPIDER_DB_PATH = "/home/jawaugh/benchmarks/raw/spider/spider/database/"

In [ ]:
# def download_file_from_google_drive(id, destination):
#     URL = "https://docs.google.com/uc?export=download"

#     session = requests.Session()

#     response = session.get(URL, params = { 'id' : id }, stream = True)
#     token = get_confirm_token(response)

#     if token:
#         params = { 'id' : id, 'confirm' : token }
#         response = session.get(URL, params = params, stream = True)

#     save_response_content(response, destination)    

# def get_confirm_token(response):
#     for key, value in response.cookies.items():
#         if key.startswith('download_warning'):
#             return value

#     return None

# def save_response_content(response, destination):
#     CHUNK_SIZE = 32768

#     with open(destination, "wb") as f:
#         for chunk in response.iter_content(CHUNK_SIZE):
#             if chunk: # filter out keep-alive new chunks
#                 f.write(chunk)

# download_file_from_google_drive("1m_M2VOjM7Xxq0z9hxgGdYIGIKhxOIU-F", "spider_train.json")
# download_file_from_google_drive("1twy32bdOYcTY8HXrISm1vTBMMjTzI9Fa", "spider_eval.json")

data = json.load(open("spider_train.json"))
for i, row in enumerate(data):
    row['row_id'] = f"spider_train|{i}"

data_eval = json.load(open("spider_eval.json"))
for i, row in enumerate(data):
    row['row_id'] = f"spider_eval|{i}"

In [ ]:
data_eval[0].keys()

In [ ]:
# compare executed results

def get_result(db_name, sql):
    conn = sqlite3.connect(os.path.join(SPIDER_DB_PATH, f"{db_name}/{db_name}.sqlite"))
    conn.text_factory = lambda b: b.decode(errors="ignore")
    cur = conn.cursor()
    cur.execute(sql)
    return cur.fetchall()

In [ ]:
async def get_database_context(db_name):
    db_path = os.path.join(SPIDER_DB_PATH, f"{db_name}/{db_name}.sqlite")
    conn = sqlite3.connect(db_path)
    c = conn.cursor()
    # execute a sql query to get all the tables and the columns in the tables
    c.execute("SELECT name FROM sqlite_master WHERE type='table';")
    outputschema = ""
    # outputschema = "Example: table.column* (row1 value) *means join key\n"
    for (table,) in c.fetchall():
        # # # STYLE 2
        # outputschema += f"===== {table} =====\n"
        # c.execute(f"PRAGMA table_info({table})")
        # column_names = [column for _, column, dtype, *_ in c.fetchall()]
        # output = c.execute(f"select * from {table} limit 2").fetchall()
        # for i, column in enumerate(column_names):
        #     if len(output) == 0:
        #         outputschema += f"{column} : -empty-"
        #     elif len(output) == 1:
        #         outputschema += f"{column} : {output[0][i]}"
        #     else:
        #         outputschema += f"{column} : {output[0][i]} | {output[1][i]}"
        #     outputschema += "\n"

        # # STYLE 1
        outputschema += f"{table} ("
        fks = [(tab, fro, to) for _, _, tab, fro, to, *_ in c.execute(f"PRAGMA foreign_key_list({table})").fetchall()]
        cols_to_star = [co for _, co, _ in fks]
        c.execute(f"PRAGMA table_info({table})")
        columns = [column + ("*" if pk or column in cols_to_star else "") for _, column, dtype, _, _, pk in c.fetchall() ]
        outputschema += "|".join(columns)
        outputschema += ") \n"
        output = c.execute(f"select * from {table} limit 2").fetchall()
        if len(output) == 0:
            outputschema += "Empty Table"
        else:
            for row in output:
                outputschema += "|".join([str(x) for x in row]) + "\n"
        outputschema += "  foreign keys: " + ", ".join([f"{fro} -> {to} ({tab})" for tab, fro, to in fks]) + "\n"
        outputschema += "\n"

        # # STYLE 3
        # fks = [(tab, fro, to) for _, _, tab, fro, to, *_ in c.execute(f"PRAGMA foreign_key_list({table})").fetchall()]
        # cols_to_star = [co for _, co, _ in fks]
        # columns = [column + ("*" if pk or column in cols_to_star else "") for _, column, dtype, _, _, pk in c.execute(f"PRAGMA table_info({table})").fetchall() ]
        # output = c.execute(f"select * from {table} limit 2").fetchall()
        # outputschema += f"== {table} ==\n"
        # for i, column in enumerate(columns):
        #     if len(output) == 0:
        #         outputschema += f"{column} ()"
        #     elif len(output) == 1:
        #         outputschema += f"{column} ({output[0][i]})"
        #     else:
        #         outputschema += f"{column} ({output[0][i]}|{output[1][i]})"
        #     outputschema += "\n"
        # outputschema += "\n"
    return outputschema

get_db_context = asyncPrompt("get_database_context", get_database_context)
correct_sqlite_error = asyncGPT3Edit(
    "correct_sqlite_error",
    "Update the query to fix the error (correct the sql query between the ``` marks): [{{ error }}]",
    model_name="code-davinci-edit-001",
    temperature=0.0,
    )

# gpt3_zeroshot_sql = asyncGPT3Prompt("gpt3_zeroshot_sql", """
# Database Context
# {{ db_context }}
# SQLite Query for question [{{ question }}]:
# ```
# """, stop="```", temperature=0.0)

# gpt3_zeroshot_explain_plan = asyncGPT3Prompt("gpt3_zeroshot_explain_plan", """
# Database Context
# {{ db_context }}
# SQLite Query for question [{{ question }}]...
# 1) First we need to consider
# """, stop="```", temperature=1.0, model_name="text-davinci-002")

gpt3_zeroshot_sql_warm = asyncGPT3Prompt("gpt3_zeroshot_sql_warm", """
Database Context
{{ db_context }}
SQLite Query for question [{{ question }}]:
```""", stop="```", temperature=0.4, model_name="code-davinci-002")

def clean_sql_quick(sql):
    sql = sql.strip()
    sql = sql.replace("\n", "")
    removals = ["sqlite3", "sqlite", "sql", "SQL"]
    for removal in sorted(removals, key=lambda x: len(x), reverse=True):
        if sql.startswith(removal):
            sql = sql[len(removal):]
    return sql

async def get_sql_for_question_multi(db_name, text_in, top_n=5):
    db_context = await get_db_context(db_name)
    sqls = await asyncio.gather(*[gpt3_zeroshot_sql_warm(db_context=db_context, question=text_in) for _ in range(top_n)])
    sqls = list(set([clean_sql_quick(sql) for sql in sqls]))

    async def run_sql_for_result(sql):
        trials = 0
        while trials < 4:
            try:
                return (sql, get_result(db_name, sql))
            except Exception as e:
                last_prompt = gpt3_zeroshot_sql_warm.get_prompt(db_context=db_context, question=text_in) + sql + '```'
                response = await correct_sqlite_error(error=str(e), input=last_prompt)
                try:
                    sql = response.split('```')[1]
                    sql = clean_sql_quick(sql)
                except:
                    sql = ""
                trials += 1
        return (sql, None)
        
    results = await asyncio.gather(*[run_sql_for_result(sql) for sql in sqls])

    answers = {json.dumps(res): set() for _, res in results if res is not None}
    for sql, res in results:
        if res is not None and sql:
            answers[json.dumps(res)].add(sql)
    if len(answers) == 0:
        return "FAILED TO GENERATE SQL"

    top_answers = sorted(answers.values(), key=len, reverse=True)[0]
    return next(iter(top_answers))

get_sql_from_text_multi = asyncPrompt("get_sql_from_text_multi", get_sql_for_question_multi)

In [ ]:
async def row_runner(row, verbose=False):
    sql_answers = {
        'query_answer': row['query'],
        'gpt3_answer': await get_sql_from_text_multi(row['db_id'], row['text_in']),
    }
    sql_executed = {}
    for k, query in sql_answers.items():
        newrow = k.split('_')[0]+'_data'
        try:
            sql_executed[k.split('_')[0]+'_data'] = get_result(row['db_id'], query)
        except Exception as e:
            sql_executed[k.split('_')[0]+'_data'] = str(e)

    if not isinstance(sql_executed['query_data'], list):
        print("Gold failed to execute")

    execution_equals = {}
    for k, v in sql_executed.items():
        newrow = k.replace('data', 'is_equal')
        if not isinstance(v, list):
            execution_equals[newrow] = False
        execution_equals[newrow] = v == sql_executed['query_data']

    execution_is_superset = {}
    for k, v in sql_executed.items():
        newrow = k.replace('data', 'is_superset')
        if not isinstance(v, list):
            execution_is_superset[newrow] = False
        if len(v) != len(sql_executed['query_data']):
            execution_is_superset[newrow] = False
        else:
            execution_is_superset[newrow] = all([set(x) >= set(y) for (x, y) in zip(v, sql_executed['query_data'])])
    
    execution_is_same_set_of_rows = {}
    for k, v in sql_executed.items():
        newrow = k.replace('data', 'is_same_set_of_rows')
        if not isinstance(v, list):
            execution_is_same_set_of_rows[newrow] = False
        execution_is_same_set_of_rows[newrow] = set(v) == set(sql_executed['query_data'])
    return sql_answers | sql_executed | execution_equals | execution_is_superset | execution_is_same_set_of_rows

In [ ]:
# await row_runner(data[1101])

In [ ]:
# [(i, x) for i, x in enumerate(data) if x['question'] == 'What are the first and last names of the customers with the 10 cheapest invoices?']
# thing = random.sample(data, 1)[0]
# rowid = random.randint(0, len(data))
# thing = data[rowid]

In [ ]:
# print(thing['question'])
# print(thing['db_id'])

In [ ]:
import pandas as pd
import time

results = []
for i in random.sample(range(len(data)), 400):
    thing = data[i]
    print(i, thing['question'], thing['db_id'])
    result = await row_runner(thing)
    result |= {'row_number': i, 'question': thing['question'], 'db_id': thing['db_id']}
    results.append(result)
    df = pd.DataFrame(results)
    print(df['query_is_equal'].sum())
    print(df[['gpt3_is_equal', 'gpt3_is_superset', 'gpt3_is_same_set_of_rows']].sum())
    time.sleep(10.0)
# import pandas as pd


In [ ]:
async def get_prompttext_for_id(row_id):
    db_name, text_in = data[row_id]['db_id'], data[row_id]['question']
    db_context = await get_db_context(db_name)
    return gpt3_zeroshot_sql_warm.get_prompt(db_context=db_context, question=text_in)

print(await get_prompttext_for_id(3903))

In [ ]:
# get_result("movie_1", "SELECT distinct stars from rating")

In [ ]:
get_result("game_injury", "select * from game")

In [ ]:
# get_result("hr_1", "select employees.department_id, commission_pct, department_name, count(*) from employees join departments on employees.department_id = departments.department_id  group by employees.department_id, commission_pct ")

In [ ]:
# get_result('products_gen_characteristics', "SELECT product_name FROM Products WHERE color_code != (SELECT color_code FROM Ref_Colors WHERE color_description = 'white') AND product_category_code NOT IN (SELECT product_category_code FROM Ref_Product_Categories WHERE unit_of_measure = 'Handful');")

In [ ]:
async def row_cosql_runner(row, their_answer, verbose=False):
    sql_answers = {
        'query_answer': row['query'],
        'unifiedskg_answer': their_answer,
    }
    sql_executed = {}
    for k, query in sql_answers.items():
        newrow = k.split('_')[0]+'_data'
        try:
            sql_executed[k.split('_')[0]+'_data'] = get_result(row['db_id'], query)
        except Exception as e:
            sql_executed[k.split('_')[0]+'_data'] = str(e)

    if not isinstance(sql_executed['query_data'], list):
        print("Gold failed to execute")

    execution_equals = {}
    for k, v in sql_executed.items():
        newrow = k.replace('data', 'is_equal')
        if not isinstance(v, list):
            execution_equals[newrow] = False
        execution_equals[newrow] = v == sql_executed['query_data']

    execution_is_superset = {}
    for k, v in sql_executed.items():
        newrow = k.replace('data', 'is_superset')
        if not isinstance(v, list):
            execution_is_superset[newrow] = False
        if len(v) != len(sql_executed['query_data']):
            execution_is_superset[newrow] = False
        else:
            execution_is_superset[newrow] = all([set(x) >= set(y) for (x, y) in zip(v, sql_executed['query_data'])])
    
    execution_is_same_set_of_rows = {}
    for k, v in sql_executed.items():
        newrow = k.replace('data', 'is_same_set_of_rows')
        if not isinstance(v, list):
            execution_is_same_set_of_rows[newrow] = False
        execution_is_same_set_of_rows[newrow] = set(v) == set(sql_executed['query_data'])
    return sql_answers | sql_executed | execution_equals | execution_is_superset | execution_is_same_set_of_rows

In [ ]:
import pandas as pd
import time

results = []
for i in random.sample(range(len(data_eval)), 400):
    thing = data_eval[i]
    print(i, thing['question'], thing['db_id'])
    result = await row_cosql_runner(thing)
    result |= {'row_number': i, 'question': thing['question'], 'db_id': thing['db_id']}
    results.append(result)
    df = pd.DataFrame(results)
    print(df['query_is_equal'].sum())
    # print(df[['seq_out_is_equal', 'gpt3_is_superset', 'gpt3_is_same_set_of_rows']].sum())
# import pandas as pd


In [ ]:
def get_input(row):
    return "{}; structed knowledge: {}".format(row['text_in'], row['struct_in'])

In [ ]:
# wrongs = [3237,1515,2889,5829,1739,1084,2582,267,1373,6578,3690,1372,2761,2375,6677,5306,593,6802,1735,3315]
# batch = [1223,4560,2079,4177,1819,302,1018,987,6894,2204,537,4071,191,6732,1604,1966,3620,2591,3237,1716,6848,5590,6731,2667,5040,5551,2722,5961,3714,4541,369,1620,1310,1829,3055,5657,4804,3873,4198,5174,5820,2331,3812,3726,3982,4310,4239,4289,1515,468,4356,1646,3390,1191,2250,3710,5689,2914,3853,2065,1981,3912,183,5222,4278,2889,5357,3981,4562,587,823,2187,5829,6113,795,2199,1739,5540,5371,1084,3505,1176,4536,1291,3813,2165,1341,2582,3112,3168,1974,6625,3340,267,1373,4316,5291,425,6253,780,2775,4075,4553,5480,3655,4602,6361,156,636,2674,1967,5544,5231,1370,4376,4058,1567,6394,5074,1399,4155,1225,6578,1398,1194,820,3727,5525,2691,3903,4945,3607,5759,6900,6127,3407,2875,6908,4577,5957,6623,3690,2288,5564,4985,2948,4422,5882,1103,6268,1056,2782,3308,576,6850,5121,4543,1771,6416,3859,5335,2169,6231,4041,4865,4665,338,1780,6228,2607,6688,3239,1681,697,6134,271,595,2558,6974,5924,4090,3431,4299,2030,1894,1372,2761,2375,4346,4950,4106,2033,6677,6938,5611,5306,3976,3449,2415,3205,6447,5723,4370,1294,1744,3574,6965,1408,5939,828,4678,842,4043,2531,2778,1698,3460,1501,593,6802,5880,305,4961,5788,3990,963,703,1735,5879,344,618,446,1181,2525,6941,3315]
inputs = []
for i in random.sample(range(len(data_eval)), 25):
    inputs.append((i, get_input(data_eval[i])))

In [ ]:
inputs

In [ ]:
outputs = [(573,
  'select t1.transcript_date, t1.transcript_id from transcripts as t1 join transcript_contents as t2 on t1.transcript_id = t2.transcript_id group by t1.transcript_id order by count(*) asc limit 1'),
 (653, 'select avg(employers) from poker_player'),
 (57,
  'select distinct t1.fname from student as t1 join has_pet as t2 on t1.stuid = t2.stuid join pets as t3 on t2.petid = t3.petid where t3.pettype = "cat" or t3.pettype = "dog"'),
 (449, 'select count(*) from matches where year = 2013 or year = 2016'),
 (270,
  'select manager_name, district from shop order by number_products desc limit 1'),
 (288,
  'select count(*), t2.name from hiring as t1 join shop as t2 on t1.shop_id = t2.shop_id group by t1.shop_id'),
 (782,
  'select code2 from country where language!= "English" and governmentform!= "Republic"'),
 (859,
  'select count(*) from orchestra where major_record_format = "CD" or major_record_format = "DVD"'),
 (680, 'select name, birth_date from people order by name'),
 (541,
  'select student_id, first_name, middle_name, last_name, count(*) from student_enrolment group by student_id order by count(*) desc limit 1'),
 (1003, 'select name from singer order by net_worth_millions asc'),
 (683,
  'select name from people where people_id not in (select people_id from poker_player)'),
 (527, 'select section_name from sections order by section_name desc'),
 (38,
  'select t2.name from concert as t1 join singer_in_concert as t2 on t1.singer_id = t2.singer_id where t1.year = 2014'),
 (766,
  'select sum(population) from country where countrycode not in (select countrycode from countrylanguage where language = "English")'),
 (5,
  'select avg(age), min(age), max(age) from singer where country = "France"'),
 (385, 'select age, hometown from teacher'),
 (613, 'select episode from tv_series order by rating'),
 (694,
  'select t1.contestant_number, t1.contestant_name from contestants as t1 join votes as t2 on t1.contestant_number = t2.contestant_number group by t2.contestant_number having count(*) >= 2'),
 (466,
  'select t1.winner_name from winner as t1 join matches as t2 on t1.winner_ioc = t2.winner_ioc join tourney_name as t3 on t2.tournament_name = t3.tournament_name where t3.turnier_name = "Australian Open" order by t1.winner_rank_points desc limit 1'),
 (430, 'select count(*) from players'),
 (940,
  'select t1.owner_id, t1.zip_code from owners as t1 join dogs as t2 on t1.owner_id = t2.owner_id group by t1.owner_id order by sum(t2.charge_amount) desc limit 1'),
 (62,
  'select major, age from student where stuid not in (select petid from pets where pettype = "cat")'),
 (98,
  'select model from cars_data where weight  < (select avg(weight) from cars_data)'),
 (919,
  'select state from owners union select t1.state from owners as t1 join professionals as t2 on t1.owner_id = t2.professional_id')]

In [ ]:
results = []
for iv, sql in outputs:
    thing = data_eval[iv]
    result = await row_cosql_runner(thing, sql)
    result |= {'row_number': iv, 'question': thing['question'], 'db_id': thing['db_id']}
    results.append(result)
    df = pd.DataFrame(results)
    print(df['query_is_equal'].sum())

In [ ]:
# import pandas as pd
# import time

# results = []
# for i in random.sample(range(len(data_eval)), 400):
#     thing = data_eval[i]
#     print(i, thing['question'], thing['db_id'])
#     result = await row_cosql_runner(thing)
#     result |= {'row_number': i, 'question': thing['question'], 'db_id': thing['db_id']}
#     results.append(result)
#     df = pd.DataFrame(results)
#     print(df['query_is_equal'].sum())

In [ ]:
df.to_excel("unifiedskg_dataeval_25.xlsx")

In [ ]:
(df.gpt3_is_superset.sum()+(df.gpt3_is_same_set_of_rows.sum()-df.gpt3_is_equal.sum())) / len(df)

In [ ]:
df.to_excel("trial8.xlsx")

In [ ]:
# !pip install openpyxl